In [2]:
%pip install transformers tokenizers datasets evaluate accelerate

In [3]:
urls = [
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0039.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_1860_dekabristy.shtml",
]


In [4]:
import html
import re

import requests


def download(url):
    return requests.get(url).text


# code borrowed from here: https://github.com/pallets/markupsafe/blob/0.23/markupsafe/__init__.py#L21
striptags_re = re.compile(r"(<!--.*?-->|<[^>]*>)")
entity_re = re.compile(r"&([^;]+);")


def to_text(s):
    return html.unescape(striptags_re.sub("", s))


def beautify(s):
    lines = [x.strip() for x in s.split("\n") if x.strip() != ""]
    for i in range(min(100, len(lines))):
        if lines[i] == "-->":
            break
    return "\n".join(lines[i + 1 :] if i < 100 else lines)


with open("dataset.txt", "w", encoding="utf-8") as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text + "\n\n")



In [5]:
import tokenizers as tok
import transformers as tr

In [6]:
tokenizer = tok.Tokenizer(tok.models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
trainer = tok.trainers.BpeTrainer(special_tokens=["[PAD]"])
tokenizer.train(["dataset.txt"], trainer)
tokenizer.enable_padding()

In [7]:
vocab = tokenizer.get_vocab()
ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
len(vocab)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


30000

In [8]:
config = tr.GPT2Config(
    vocab_size=len(vocab),
    bos_token_id=tokenizer.token_to_id("[CLS]"),
    eos_token_id=tokenizer.token_to_id("[EOS]"),
)
gpt = tr.GPT2LMHeadModel(config)

In [9]:
res = gpt.generate(
    **ttokenizer("Мне нравится ", return_tensors="pt"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
ttokenizer.decode(res[0])

'Мне нравится ма высокое ревно Купцы Купцы печально тума тума тума слушать старичку ничества печально широкие широкие широкие широкие широкие широкие подтвердила влюблена черед веселя Посмотри Посмотри Посмотри Посмотри боюсь дость ющих ристы ристы нижняя невз дость дость дость дость сделанный сделанный печально печально находимся грубость черед дость тарелки дость дость дость'

In [10]:
import datasets

dataset = datasets.load_dataset("text", data_files="dataset.txt")
dataset["train"][13]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'Он взял своею большою рукой меня за руку, и пожал так крепко, честно, только что не больно. Я думала, что он поцелует мою руку, и нагнулась было к нему, но он еще раз пожал мне руку и прямо в глаза посмотрел своим твердым и веселым взглядом.'}

In [11]:
def tokenize(x):
    x = ttokenizer(x["text"])
    x["labels"] = x["input_ids"].copy()
    return x


ds = dataset.map(tokenize, batched=True, remove_columns=["text"])
ds["train"][0]

Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

{'input_ids': [9585, 9736, 3192],
 'token_type_ids': [0, 0, 0],
 'attention_mask': [1, 1, 1],
 'labels': [9585, 9736, 3192]}

In [14]:
from itertools import chain

block_size = 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dsb = ds.map(group_texts, batched=True)


Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

In [15]:
targs = tr.TrainingArguments(
    output_dir="gpt2-scratch",
    num_train_epochs=30,
    learning_rate=5e-5,
    warmup_steps=200,
    save_steps=1500,
)
trainer = tr.Trainer(
    gpt,
    args=targs,
    train_dataset=dsb["train"],
    tokenizer=ttokenizer,
    data_collator=tr.default_data_collator,  # tr.DataCollatorForLanguageModeling(tokenizer=ttokenizer,mlm=False)
)

In [16]:
res = gpt.generate(
    **ttokenizer("Пьер закашлялся и", return_tensors="pt"),
    max_new_tokens=150,
    do_sample=True
)
ttokenizer.decode(res[0])

KeyboardInterrupt: 

In [17]:
tokenizer = tr.AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
gpt = tr.GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
res = gpt.generate(
    **tokenizer("Мне нравится, что вы ", return_tensors="pt"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
tokenizer.decode(res[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'Мне нравится, что вы  не  хотите  меня  видеть,  но  я  не  хочу  видеть  вас  в  таком  состоянии \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [18]:
dataset = datasets.load_dataset("text", data_files="dataset.txt")
ds = dataset.map(lambda x:
                 tokenizer(x["text"]), batched=True, remove_columns=["text"])
dsb = ds.map(group_texts, batched=True)

Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

In [1]:
targs = tr.TrainingArguments(
    output_dir="gpt2-finetune",
    num_train_epochs=30,
    learning_rate=5e-5,
    warmup_steps=200,
    save_steps=1500,
)
trainer = tr.Trainer(
    gpt,
    args=targs,
    train_dataset=dsb["train"],
    tokenizer=tokenizer,
    data_collator=tr.default_data_collator,  # tr.DataCollatorForLanguageModeling(tokenizer=ttokenizer,mlm=False)
)
trainer.train()

NameError: name 'tr' is not defined

In [ ]:
res = gpt.generate(
    **tokenizer("Мне нравится, что вы ", return_tensors="pt").to("cuda"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
tokenizer.decode(res[0])